In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Embedding, SpatialDropout1D, LSTM, GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Activation, Dropout
# import LabelEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from matplotlib import pyplot as plt

from imblearn.over_sampling import SMOTE
from collections import Counter

from sklearn.metrics import classification_report, confusion_matrix
#test train split
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter

from tensorflow.keras.layers import Bidirectional, LSTM


/Users/salmazainana/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Preprocess data 

In [3]:
df = pd.read_csv('cleaned_dataset.csv')
df.columns

Index(['type', 'posts', 'cleaned_posts'], dtype='object')

In [4]:
def graph_plots(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.savefig(string + '.png')
  plt.show()

In [5]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df['cleaned_posts'])
tokenizer.word_index # Get our learned vocabulary
word_index = tokenizer.word_index

VOCAB_SIZE = len(word_index)+1 # Total words


In [6]:
X = tokenizer.texts_to_sequences(df['cleaned_posts'])
MAX_SEQ_LENGTH = max(len(seq) for seq in X)
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen= MAX_SEQ_LENGTH, padding = 'post') # Pad the sequence to the same length to make it uniform

In [7]:
from gensim.models import Word2Vec, KeyedVectors

gn_vec_zip_path ="/Users/salmazainana/Downloads/GoogleNews-vectors-negative300.bin"
model = KeyedVectors.load_word2vec_format(gn_vec_zip_path, binary=True)


In [8]:
word2vec_model = model

In [9]:
embedding_size = 300
embedding_matrix = np.zeros((VOCAB_SIZE, embedding_size))
for word, i in tokenizer.word_index.items():
    if word in word2vec_model:  # Use word2vec_model directly instead of word2vec_model.wv
        embedding_vector = word2vec_model[word]  # Same here
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [10]:
y = df['type']
#one hote encode the target variable such that ENTJ = [1,0,0,0], INTJ = [0,0,0,0], ENTP = [1,0,0,1], ect 
#dichotomy classification
# strip letter from the type so ['ENTJ'] becomes ['E','N','T','J']
y_stripped = y.apply(lambda x: list(x.strip()))

def preprocessing(y):
    if y[0] == 'E':
        y[0] = 1
    else:
        y[0] = 0
    if y[1] == 'N':
        y[1] = 1
    else:
        y[1] = 0
    if y[2] == 'T':
        y[2] = 1
    else:
        y[2] = 0
    if y[3] == 'J':
        y[3] = 1
    else:
        y[3] = 0

    return y 

y = y_stripped.apply(preprocessing)
y = np.array(y.tolist())
type(y)


numpy.ndarray

In [11]:
# model_path = "/Users/zainanasalma/Desktop/stanford/courses/CS229/mbti-cs229/Word2vec_model.model"
# #load model from model_path
# word2vec_model = tf.keras.models.load_model(model_path)

embedding_size = 300
# embedding_matrix = np.zeros((VOCAB_SIZE, embedding_size))

# for word, i in tokenizer.word_index.items():
#     if word in word2vec_model.wv:
#         embedding_vector = word2vec_model.wv[word]
#         if embedding_vector is not None:
#             embedding_matrix[i] = embedding_vector




In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
# Assuming y_train is a numpy array of integer labels
# Convert integer labels to one-hot encoded labels
num_classes = 4  # Adjust based on your actual number of classes

model = Sequential()
model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=embedding_size, weights=[embedding_matrix], input_length=MAX_SEQ_LENGTH, trainable=False))

model.add(Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
model.add(Dropout(0.8))

model.add(Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
model.add(Dropout(0.8))

model.add(Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences=False)))  # Ensure the last LSTM layer does not return sequences
model.add(Dropout(0.8))

model.add(Dense(4, activation='relu'))  # Intermediate layer (if needed)
model.add(Dense(num_classes, activation='softmax'))  # Output layer for 4 classes

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

model.summary()

# Fit the model
# Replace y_train with y_train_one_hot in the model.fit call
history = model.fit(X_train, y_train, validation_split=0.2, epochs=30, batch_size=128, callbacks=[EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 957, 300)          40590900  
                                                                 
 bidirectional (Bidirection  (None, 957, 512)          1140736   
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 957, 512)          0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 957, 512)          1574912   
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 957, 512)          0         
                                                                 
 bidirectional_2 (Bidirecti  (None, 512)               1

: 

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc

y_pred = model.predict(X_test)
n_classes = y_test.shape[1]  # Make sure n_classes is defined

# Calculate AUC for each binary classification task and plot ROC curves
auc_scores = []

for i in range(n_classes):
    score = roc_auc_score(y_test[:, i], y_pred[:, i])  # Use a different variable name
    auc_scores.append(score)
    print(f"AUC for task {i}: {score}")

    fpr, tpr, thresholds = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc = auc(fpr, tpr)  
    plt.plot(fpr, tpr, label=f'Task {i+1} (area = {roc_auc:.2f})')


plt.plot([0, 1], [0, 1], 'k--', label='Random chance')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend(loc="lower right")
plt.show()

# Calculate the average AUC across all tasks
average_auc = np.mean(auc_scores)
print(f"Average AUC: {average_auc}")

In [ ]:
def find_best_threshold(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    # Calculate the J statistic for each threshold
    J = tpr - fpr
    # Find the index of the maximum J statistic
    index_of_max_J = np.argmax(J)
    best_threshold = thresholds[index_of_max_J]
    return best_threshold, J[index_of_max_J]

best_threshold= np.zeros(n_classes)
for i in range(n_classes):
    best_threshold[i], max_J = find_best_threshold(y_test[:, i], y_pred[:, i])
    print(f"Best threshold for task {i}: {best_threshold} with J statistic: {max_J}")


In [ ]:
#score accuracy
for i in range(n_classes):
    # return 1 if y_pred[i] > best_threshold[i], else 0
    if i == 2: 
        y_pred[:, i] = (y_pred[:, i] < best_threshold[i]).astype(int)
    else :
        y_pred[:, i] = (y_pred[:, i] > best_threshold[i]).astype(int)
    
    print(f"Accuracy for task {i}: {accuracy_score(y_test[:, i], y_pred[:, i])}")